## Kaggle 2 
### By - Harshalkumar Loya & Harshal Talele

In [3]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.0/22.0 MB 24.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 7.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, KBinsDiscretizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
import category_encoders as ce
from sklearn.impute import KNNImputer
from sklearn.model_selection import GridSearchCV
import warnings
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor


In [5]:
train_data = pd.read_csv('training_data_kaggle2.csv', index_col = 1, low_memory=False)
test_data = pd.read_csv('test_data_kaggle-2.csv', index_col = 1, low_memory = False)
test_data.drop('winner_ratio', axis=1, inplace=True)


In [6]:
train_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

In [7]:
train_data['us_senate_contribution_ratio'] = train_data['us_senate_contribution_ratio'].replace('#NAME?', np.nan).astype('float64')
train_data['senate_contribution_ratio'] = train_data['senate_contribution_ratio'].replace('#NAME?', np.nan).astype('float64')
train_data['governor_contribution_ratio'] = train_data['governor_contribution_ratio'].replace('#NAME?', np.nan).astype('float64')


In [8]:
train_data = train_data.replace([np.inf, -np.inf], np.nan)
test_data = test_data.replace([np.inf, -np.inf], np.nan)


In [9]:
# Creating an instance of the KNNImputer
imputer = KNNImputer(n_neighbors=5)

# Getting only numerical features
numerical_columns = train_data.select_dtypes(include=[np.number]).columns

# Fitting the imputer and transforming the numerical data
train_data_numerical_imputed = imputer.fit_transform(train_data[numerical_columns])

# Converting the imputed numerical data back to a DataFrame
train_data_numerical_imputed = pd.DataFrame(train_data_numerical_imputed, columns=numerical_columns)

# Replacing the original numerical columns in the dataset with the imputed ones
train_data[numerical_columns] = train_data_numerical_imputed


In [10]:
# Getting only numerical features
numerical_columns = test_data.select_dtypes(include=[np.number]).columns

# Fitting the imputer and transforming the numerical data
test_data_numerical_imputed = imputer.fit_transform(test_data[numerical_columns])

# Converting the imputed numerical data back to a DataFrame
test_data_numerical_imputed = pd.DataFrame(test_data_numerical_imputed, columns=numerical_columns)

# Replacing the original numerical columns in the dataset with the imputed ones
test_data[numerical_columns] = test_data_numerical_imputed

In [11]:
numerical_features = train_data.select_dtypes(include=[np.number]).columns.tolist()
print(numerical_features)


['zip_code', 'winner_ratio', 'candidacy_count', 'candidacy_democratic_count', 'candidacy_republican_count', 'contribution_count', 'contribution_democratic_count', 'contribution_republican_count', 'politician_challenger_count', 'politician_count', 'politician_democratic_count', 'politician_incumbency_count', 'politician_open_pos_count', 'politician_republican_count', 'contribution_democratic_sum_2010_usd', 'contribution_republican_sum_2010_usd', 'contribution_sum_2010_usd', 'governor_contributions_sum_2010_usd', 'house_and_assembly_contributions_sum_2010_usd', 'senate_contributions_sum_2010_usd', 'us_house_contributions_sum_2010_usd', 'us_senate_contributions_sum_2010_usd', 'candidacy_democratic_ratio', 'candidacy_republican_ratio', 'contribution_democratic_count_ratio', 'contribution_republican_count_ratio', 'governor_contribution_ratio', 'house_and_assembly_contribution_ratio', 'politician_challenger_ratio', 'politician_democratic_ratio', 'politician_incumbency_ratio', 'politician_ope

In [12]:
'''
# Split the dataset into state and federal level data
train_data_state = train_data[train_data['state'] != 'US']
train_data_federal = train_data[train_data['state'] == 'US']

test_data_state = test_data[test_data['state'] != 'US']
test_data_federal = test_data[test_data['state'] == 'US']

print(train_data_state.shape)
print(test_data_state.shape)
print(train_data_federal.shape)
print(test_data_federal.shape)
'''

"\n# Split the dataset into state and federal level data\ntrain_data_state = train_data[train_data['state'] != 'US']\ntrain_data_federal = train_data[train_data['state'] == 'US']\n\ntest_data_state = test_data[test_data['state'] != 'US']\ntest_data_federal = test_data[test_data['state'] == 'US']\n\nprint(train_data_state.shape)\nprint(test_data_state.shape)\nprint(train_data_federal.shape)\nprint(test_data_federal.shape)\n"

In [13]:
def label_election(row):
    if row['us_senate_contribution_ratio'] == 1:
        return 'US_senate'
    elif row['us_house_contribution_ratio'] == 1:
        return 'US_house'
    elif row['senate_contribution_ratio'] == 1:
        return 'senate'
    elif row['governor_contribution_ratio'] == 1:
        return 'governor'
    elif row['house_and_assembly_contribution_ratio'] == 1:
        return 'house_and_assembly'
    else:
        return 'other'

# Assuming your data is in a DataFrame called df
train_data['election_type'] = train_data.apply(label_election, axis=1)
test_data['election_type'] = test_data.apply(label_election, axis=1)


In [14]:
train_data_USSenate = train_data[train_data['election_type'] == 'US_senate']
train_data_USHouse = train_data[train_data['election_type'] == 'US_house']
train_data_Senate = train_data[train_data['election_type'] == 'senate']
train_data_Governor = train_data[train_data['election_type'] == 'governor']
train_data_HouseAssembly = train_data[train_data['election_type'] == 'house_and_assembly']
train_data_mixed = train_data[train_data['election_type'] == 'other']


test_data_USSenate = test_data[test_data['election_type'] == 'US_senate']
test_data_USHouse = test_data[test_data['election_type'] == 'US_house']
test_data_Senate = test_data[test_data['election_type'] == 'senate']
test_data_Governor = test_data[test_data['election_type'] == 'governor']
test_data_HouseAssembly = test_data[test_data['election_type'] == 'house_and_assembly']
test_data_mixed = test_data[test_data['election_type'] == 'other']

In [15]:
# List of traditionally red and blue states
traditionally_red_states = ["FL","AL", "AK", "AZ", "AR", "GA", "ID", "KS", "KY", "LA", "MS", "MT", "NE", "ND", "OK", "SC", "SD", "TN", "TX", "UT", "WV", "WY"]
traditionally_blue_states = ["CA", "CT", "DE", "HI", "IL", "ME", "MD", "MA", "MN", "NH", "NJ", "NM", "NY", "OR", "RI", "VT", "WA"]

# Add a new column to the dataset indicating the traditional political color
def state_color(state):
    if state in traditionally_red_states:
        return 'red'
    elif state in traditionally_blue_states:
        return 'blue'
    else:
        return 'purple'

train_data_Senate['state_color'] = train_data['state'].apply(state_color)
test_data_Senate['state_color'] = test_data['state'].apply(state_color)

train_data_Governor['state_color'] = train_data['state'].apply(state_color)
test_data_Governor['state_color'] = test_data['state'].apply(state_color)

train_data_HouseAssembly['state_color'] = train_data['state'].apply(state_color)
test_data_HouseAssembly['state_color'] = test_data['state'].apply(state_color)

train_data_mixed['state_color'] = train_data['state'].apply(state_color)
test_data_mixed['state_color'] = test_data['state'].apply(state_color)

train_data_USSenate['state_color'] = train_data['state'].apply(state_color)
test_data_USSenate['state_color'] = test_data['state'].apply(state_color)

train_data_USHouse['state_color'] = train_data['state'].apply(state_color)
test_data_USHouse['state_color'] = test_data['state'].apply(state_color)

/var/folders/ms/41jtny6x4s923y4j0q_lr68w0000gn/T/ipykernel_49377/3987487817.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_Senate['state_color'] = train_data['state'].apply(state_color)
/var/folders/ms/41jtny6x4s923y4j0q_lr68w0000gn/T/ipykernel_49377/3987487817.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data_Senate['state_color'] = test_data['state'].apply(state_color)
/var/folders/ms/41jtny6x4s923y4j0q_lr68w0000gn/T/ipykernel_49377/3987487817.py:17: SettingWithCopyWarning: 


In [16]:
# Define a function to preprocess, train, and predict using the given dataset
def preprocess_train_predict(train_data, test_data, selected_features, model):
    # Select only specified features
    train_data = train_data[selected_features + ['winner_ratio']]

    # Preprocess data
    num_features = train_data.select_dtypes(include=[np.number]).columns.tolist()
    cat_features = train_data.select_dtypes(include=[object]).columns.tolist()
    num_features.remove('winner_ratio')

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), num_features),
            ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)])

    # Split data
    X = train_data.drop(columns=['winner_ratio'])
    y = train_data['winner_ratio']

    # Create a pipeline
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('model', model)])
    # Train model
    pipeline.fit(X, y)

    # Predict
    X_test = test_data[selected_features]
    test_pred  = pipeline.predict(X_test)
    return test_pred

In [17]:

# Split data
X = train_data.drop(columns=['winner_ratio'])
y = train_data['winner_ratio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)
X_train_encoded = pd.get_dummies(X_train, columns=['general_sector', 'specific_sector', 'state', 'contributor_type'])


In [18]:
'''
# Create a XGBRegressor model for tuning

tuning_model = XGBRegressor(objective='reg:squarederror', n_jobs=-1, random_state=42)

grid_search = GridSearchCV(estimator=tuning_model, param_grid=param_grid, cv=3, scoring='neg_root_mean_squared_error', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

'''

"\n# Create a XGBRegressor model for tuning\n\ntuning_model = XGBRegressor(objective='reg:squarederror', n_jobs=-1, random_state=42)\n\ngrid_search = GridSearchCV(estimator=tuning_model, param_grid=param_grid, cv=3, scoring='neg_root_mean_squared_error', n_jobs=-1, verbose=2)\ngrid_search.fit(X_train, y_train)\n\n"

In [19]:
#best_params = grid_search.best_params_

#print("Best hyperparameters for tuning model:", best_params)


In [20]:
#model1 = RandomForestRegressor(n_estimators=100, random_state=42)
#model2 = RandomForestRegressor(n_estimators=100, random_state=42)

selected_numerical_features_USSenate = ['candidacy_count', 'candidacy_democratic_count', 'candidacy_republican_count', 'contribution_count', 'contribution_democratic_count', 'contribution_republican_count', 'politician_challenger_count', 'politician_count', 'politician_democratic_count', 'politician_incumbency_count', 'politician_open_pos_count', 'politician_republican_count', 'contribution_democratic_sum_2010_usd', 'contribution_republican_sum_2010_usd', 'contribution_sum_2010_usd', 'us_house_contributions_sum_2010_usd', 'us_senate_contributions_sum_2010_usd', 'candidacy_democratic_ratio', 'contribution_democratic_count_ratio', 'governor_contribution_ratio', 'house_and_assembly_contribution_ratio', 'politician_challenger_ratio', 'politician_democratic_ratio', 'politician_incumbency_ratio', 'politician_open_pos_ratio', 'us_house_contribution_ratio', 'us_senate_contribution_ratio']
selected_numerical_features_USHouse = ['candidacy_count', 'candidacy_democratic_count', 'candidacy_republican_count', 'contribution_count', 'contribution_democratic_count', 'contribution_republican_count', 'politician_challenger_count', 'politician_count', 'politician_democratic_count', 'politician_incumbency_count', 'politician_open_pos_count', 'politician_republican_count', 'contribution_democratic_sum_2010_usd', 'contribution_republican_sum_2010_usd', 'contribution_sum_2010_usd', 'us_house_contributions_sum_2010_usd', 'us_senate_contributions_sum_2010_usd', 'candidacy_democratic_ratio', 'contribution_democratic_count_ratio', 'governor_contribution_ratio', 'house_and_assembly_contribution_ratio', 'politician_challenger_ratio', 'politician_democratic_ratio', 'politician_incumbency_ratio', 'politician_open_pos_ratio', 'us_house_contribution_ratio', 'us_senate_contribution_ratio']
selected_numerical_features_Senate = [ 'candidacy_count', 'candidacy_democratic_count', 'candidacy_republican_count', 'contribution_count', 'contribution_democratic_count', 'contribution_republican_count', 'politician_challenger_count', 'politician_count', 'politician_democratic_count', 'politician_incumbency_count', 'politician_open_pos_count', 'politician_republican_count', 'contribution_democratic_sum_2010_usd', 'contribution_republican_sum_2010_usd', 'contribution_sum_2010_usd', 'governor_contributions_sum_2010_usd', 'house_and_assembly_contributions_sum_2010_usd', 'senate_contributions_sum_2010_usd', 'us_house_contributions_sum_2010_usd', 'us_senate_contributions_sum_2010_usd', 'candidacy_democratic_ratio', 'contribution_democratic_count_ratio', 'governor_contribution_ratio', 'house_and_assembly_contribution_ratio', 'politician_challenger_ratio', 'politician_democratic_ratio', 'politician_incumbency_ratio', 'politician_open_pos_ratio', 'senate_contribution_ratio', 'us_house_contribution_ratio', 'us_senate_contribution_ratio']
selected_numerical_features_Governor = [ 'candidacy_count', 'candidacy_democratic_count', 'candidacy_republican_count', 'contribution_count', 'contribution_democratic_count', 'contribution_republican_count', 'politician_challenger_count', 'politician_count', 'politician_democratic_count', 'politician_incumbency_count', 'politician_open_pos_count', 'politician_republican_count', 'contribution_democratic_sum_2010_usd', 'contribution_republican_sum_2010_usd', 'contribution_sum_2010_usd', 'governor_contributions_sum_2010_usd', 'house_and_assembly_contributions_sum_2010_usd', 'senate_contributions_sum_2010_usd', 'us_house_contributions_sum_2010_usd', 'us_senate_contributions_sum_2010_usd', 'candidacy_democratic_ratio', 'contribution_democratic_count_ratio', 'governor_contribution_ratio', 'house_and_assembly_contribution_ratio', 'politician_challenger_ratio', 'politician_democratic_ratio', 'politician_incumbency_ratio', 'politician_open_pos_ratio', 'senate_contribution_ratio', 'us_house_contribution_ratio', 'us_senate_contribution_ratio']
selected_numerical_features_HouseAssembly = [ 'candidacy_count', 'candidacy_democratic_count', 'candidacy_republican_count', 'contribution_count', 'contribution_democratic_count', 'contribution_republican_count', 'politician_challenger_count', 'politician_count', 'politician_democratic_count', 'politician_incumbency_count', 'politician_open_pos_count', 'politician_republican_count', 'contribution_democratic_sum_2010_usd', 'contribution_republican_sum_2010_usd', 'contribution_sum_2010_usd', 'governor_contributions_sum_2010_usd', 'house_and_assembly_contributions_sum_2010_usd', 'senate_contributions_sum_2010_usd', 'us_house_contributions_sum_2010_usd', 'us_senate_contributions_sum_2010_usd', 'candidacy_democratic_ratio', 'contribution_democratic_count_ratio', 'governor_contribution_ratio', 'house_and_assembly_contribution_ratio', 'politician_challenger_ratio', 'politician_democratic_ratio', 'politician_incumbency_ratio', 'politician_open_pos_ratio', 'senate_contribution_ratio', 'us_house_contribution_ratio', 'us_senate_contribution_ratio']
selected_numerical_features_mixed = [ 'candidacy_count', 'candidacy_democratic_count', 'candidacy_republican_count', 'contribution_count', 'contribution_democratic_count', 'contribution_republican_count', 'politician_challenger_count', 'politician_count', 'politician_democratic_count', 'politician_incumbency_count', 'politician_open_pos_count', 'politician_republican_count', 'contribution_democratic_sum_2010_usd', 'contribution_republican_sum_2010_usd', 'contribution_sum_2010_usd', 'governor_contributions_sum_2010_usd', 'house_and_assembly_contributions_sum_2010_usd', 'senate_contributions_sum_2010_usd', 'us_house_contributions_sum_2010_usd', 'us_senate_contributions_sum_2010_usd', 'candidacy_democratic_ratio', 'contribution_democratic_count_ratio', 'governor_contribution_ratio', 'house_and_assembly_contribution_ratio', 'politician_challenger_ratio', 'politician_democratic_ratio', 'politician_incumbency_ratio', 'politician_open_pos_ratio', 'senate_contribution_ratio', 'us_house_contribution_ratio', 'us_senate_contribution_ratio']

selected_categorocal_features_USSenate = [ 'state','state_color']
selected_categorocal_features_USHouse = ['state','state_color']
selected_categorocal_features_Senate = ['state', 'state_color']
selected_categorocal_features_Governor = ['state', 'state_color']
selected_categorocal_features_HouseAssembly = ['state', 'state_color']
selected_categorocal_features_mixed = ['state', 'state_color']


selected_features_USSenate = selected_numerical_features_USSenate + selected_categorocal_features_USSenate
selected_features_USHouse = selected_numerical_features_USHouse + selected_categorocal_features_USHouse
selected_features_Senate = selected_numerical_features_Senate + selected_categorocal_features_Senate
selected_features_Governor = selected_numerical_features_Governor + selected_categorocal_features_Governor
selected_features_HouseAssembly = selected_numerical_features_HouseAssembly + selected_categorocal_features_HouseAssembly
selected_features_mixed = selected_numerical_features_mixed + selected_categorocal_features_mixed

model1 = CatBoostRegressor(iterations=1000,depth=6,learning_rate=0.1, loss_function='RMSE',verbose=100)
model2 = CatBoostRegressor(iterations=1000,depth=6,learning_rate=0.1, loss_function='RMSE',verbose=100)
model3 = CatBoostRegressor(iterations=1000,depth=6,learning_rate=0.1, loss_function='RMSE',verbose=100)
model4 = CatBoostRegressor(iterations=1000,depth=6,learning_rate=0.1, loss_function='RMSE',verbose=100)
model5 = CatBoostRegressor(iterations=1000,depth=6,learning_rate=0.1, loss_function='RMSE',verbose=100)
model6 = CatBoostRegressor(iterations=1000,depth=6,learning_rate=0.1, loss_function='RMSE',verbose=100)
'''
model1 = Ridge(alpha=1.0)
model1 = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

model2 = Ridge(alpha=1.0)
model2 = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

model3 = Ridge(alpha=1.0)
model3 = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

model4 = Ridge(alpha=1.0)
model4 = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

model5 = Ridge(alpha=1.0)
model5 = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

model6 = Ridge(alpha=1.0)
model6 = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
'''

'\nmodel1 = Ridge(alpha=1.0)\nmodel1 = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)\n\nmodel2 = Ridge(alpha=1.0)\nmodel2 = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)\n\nmodel3 = Ridge(alpha=1.0)\nmodel3 = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)\n\nmodel4 = Ridge(alpha=1.0)\nmodel4 = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)\n\nmodel5 = Ridge(alpha=1.0)\nmodel5 = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)\n\nmodel6 = Ridge(alpha=1.0)\nmodel6 = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)\n'

In [21]:
test_pred_USSenate = preprocess_train_predict(train_data_USSenate, test_data_USSenate, selected_features_USSenate, model1)
test_pred_USHouse = preprocess_train_predict(train_data_USHouse, test_data_USHouse, selected_features_USHouse, model2)
test_pred_Senate = preprocess_train_predict(train_data_Senate, test_data_Senate, selected_features_Senate, model3)
test_pred_Governor = preprocess_train_predict(train_data_Governor, test_data_Governor, selected_features_Governor, model4)
test_pred_HouseAssembly = preprocess_train_predict(train_data_HouseAssembly, test_data_HouseAssembly, selected_features_HouseAssembly, model5)
test_pred_mixed = preprocess_train_predict(train_data_mixed, test_data_mixed, selected_features_mixed, model6)


0:	learn: 0.3959427	total: 62.4ms	remaining: 1m 2s
100:	learn: 0.3184923	total: 226ms	remaining: 2.01s
200:	learn: 0.3030582	total: 390ms	remaining: 1.55s
300:	learn: 0.2939180	total: 544ms	remaining: 1.26s
400:	learn: 0.2858173	total: 708ms	remaining: 1.06s
500:	learn: 0.2802906	total: 870ms	remaining: 867ms
600:	learn: 0.2746261	total: 1.03s	remaining: 686ms
700:	learn: 0.2707159	total: 1.2s	remaining: 511ms
800:	learn: 0.2673518	total: 1.36s	remaining: 339ms
900:	learn: 0.2636859	total: 1.52s	remaining: 168ms
999:	learn: 0.2602553	total: 1.68s	remaining: 0us
0:	learn: 0.4258175	total: 2.82ms	remaining: 2.81s
100:	learn: 0.3211762	total: 210ms	remaining: 1.87s
200:	learn: 0.3116847	total: 417ms	remaining: 1.66s
300:	learn: 0.3057957	total: 624ms	remaining: 1.45s
400:	learn: 0.3011182	total: 835ms	remaining: 1.25s
500:	learn: 0.2972623	total: 1.04s	remaining: 1.03s
600:	learn: 0.2936487	total: 1.24s	remaining: 822ms
700:	learn: 0.2904896	total: 1.45s	remaining: 620ms
800:	learn: 0.288

In [22]:
test_pred_USSenate = pd.Series(test_pred_USSenate, index=test_data_USSenate.index, name='winner_ratio')
test_pred_USHouse = pd.Series(test_pred_USHouse, index=test_data_USHouse.index, name='winner_ratio')
test_pred_Senate = pd.Series(test_pred_Senate, index=test_data_Senate.index, name='winner_ratio')
test_pred_Governor = pd.Series(test_pred_Governor, index=test_data_Governor.index, name='winner_ratio')
test_pred_HouseAssembly = pd.Series(test_pred_HouseAssembly, index=test_data_HouseAssembly.index, name='winner_ratio')
test_pred_mixed = pd.Series(test_pred_mixed, index=test_data_mixed.index, name='winner_ratio')

test_pred = pd.concat([test_pred_USSenate, test_pred_USHouse, test_pred_Senate, test_pred_Governor, test_pred_HouseAssembly, test_pred_mixed]).sort_index()
test_pred = test_pred.reset_index(drop=True)


In [23]:
# Model 1: USSenate
for i in range(len(selected_features_USSenate)):
    print(selected_features_USSenate[i], "-", model1.feature_importances_[i])

print("\n")  # Adding a new line for better readability

# Model 2: USHouse
for i in range(len(selected_features_USHouse)):
    print(selected_features_USHouse[i], "-", model2.feature_importances_[i])

print("\n")  # Adding a new line for better readability

# Model 3: Senate
for i in range(len(selected_features_Senate)):
    print(selected_features_Senate[i], "-", model3.feature_importances_[i])

print("\n")  # Adding a new line for better readability

# Model 4: Governor
for i in range(len(selected_features_Governor)):
    print(selected_features_Governor[i], "-", model4.feature_importances_[i])

print("\n")  # Adding a new line for better readability

# Model 5: HouseAssembly
for i in range(len(selected_features_HouseAssembly)):
    print(selected_features_HouseAssembly[i], "-", model5.feature_importances_[i])

print("\n")  # Adding a new line for better readability

# Model 6: mixed
for i in range(len(selected_features_mixed)):
    print(selected_features_mixed[i], "-", model6.feature_importances_[i])



candidacy_count - 1.2360950808656639
candidacy_democratic_count - 0.3608620902510312
candidacy_republican_count - 0.47023422430200285
contribution_count - 4.525993736932868
contribution_democratic_count - 2.7737792197275937
contribution_republican_count - 1.994194493644599
politician_challenger_count - 2.7689078713585347
politician_count - 0.980307698967874
politician_democratic_count - 0.5013239746756778
politician_incumbency_count - 4.487510768303428
politician_open_pos_count - 2.292211296900816
politician_republican_count - 0.9623247082091634
contribution_democratic_sum_2010_usd - 14.864028759910235
contribution_republican_sum_2010_usd - 6.72995802721423
contribution_sum_2010_usd - 15.807784276838788
us_house_contributions_sum_2010_usd - 0.0
us_senate_contributions_sum_2010_usd - 8.580551268347376
candidacy_democratic_ratio - 0.8542949402488943
contribution_democratic_count_ratio - 0.8339271753250738
governor_contribution_ratio - 0.0
house_and_assembly_contribution_ratio - 0.0
polit

In [24]:
out_csv = pd.DataFrame(test_pred, columns=['winner_ratio'])
out_csv['Index'] = out_csv.index
out_csv['winner_ratio'] = (pd.to_numeric(out_csv['winner_ratio'], errors='coerce')).astype('Float64')
out_csv = out_csv[['Index', 'winner_ratio']]
out_csv.to_csv('kaggle2_submission.csv', index=False)

In [25]:
out_csv.head(10)

,Index,winner_ratio
0,0,0.89864
1,1,0.999192
2,2,0.746817
3,3,0.538428
4,4,0.920004
5,5,0.658193
6,6,0.566202
7,7,-0.00409
8,8,0.910672
9,9,0.959094
